# **Twitter Sentiment Analysis**





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
data =pd.read_csv('/content/drive/My Drive/Twitter Sentiment Analysis/train_E6oV3lV.csv',index_col=0)
test =pd.read_csv('/content/drive/My Drive/Twitter Sentiment Analysis/test_tweets_anuFYb8.csv',index_col=0)


In [5]:
print(data.shape)
print(test.shape)

(31962, 2)
(17197, 1)


In [6]:
data.head()

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation


**Preprocessing:**

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# import nltk
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [160]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [213]:
import re
from nltk.corpus import stopwords
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
stop_words=stopwords.words('english')
def preprocessing(document):
  document=list(document)
  lemm=WordNetLemmatizer()
  for i in range(0,len(document)):
    document[i]=document[i].lower()
    document[i]=document[i].replace('user','')
    #regex to remove all the words with numbers for reference: https://stackoverflow.com/a/18082370/4084039
    document[i] = re.sub("\S*\d\S*", "", document[i]).strip()
    #remove special character: https://stackoverflow.com/a/5843547/4084039
    document[i] = re.sub('[^A-Za-z0-9]+', ' ', document[i])
    document[i]=word_tokenize(document[i])
    document[i]=[lemm.lemmatize(words) for words in document[i]]
    document[i] = ' '.join(e.lower() for e in document[i] if e.lower() not in stop_words)
  return document

In [214]:
nltk.download('wordnet')
nltk.download('punkt')
from nltk import word_tokenize
preprocessed_train=preprocessing(train['tweet'])
preprocessed_test=preprocessing(test['tweet'])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [216]:
preprocessed_train[:2000]

['father dysfunctional selfish drag kid dysfunction run',
 'thanks lyft credit use cause offer wheelchair van pdx disapointed getthanked',
 'bihday majesty',
 'model love u take u time ur',
 'factsguide society motivation',
 'huge fan fare big talking leave chaos pay dispute get allshowandnogo',
 'camping tomorrow danny',
 'next school year year exam think school exam hate imagine actorslife revolutionschool girl',
 'love land allin cavs champion cleveland clevelandcavaliers',
 'welcome',
 'ireland consumer price index mom climbed previous may blog silver gold forex',
 'selfish orlando standwithorlando pulseshooting orlandoshooting biggerproblems selfish heabreaking value love',
 'get see daddy today gettingfed',
 'cnn call michigan middle school build wall chant tcot',
 'comment australia opkillingbay seashepherd helpcovedolphins thecove helpcovedolphins',
 'ouch junior junior yugyoem omg',
 'thankful paner thankful positive',
 'retweet agree',
 'friday smile around via ig cooky make 

In [44]:
train.shape,len(preprocessed_train)

((31962, 2), 31962)

**Using Unigram, Bigram, Trigram and tetragrams in TFIDF Vectorizer**

In [230]:
Y=train['label']
X=preprocessed_train
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.33,stratify=Y)
#print(X_train.shape,X_test.shape)

from sklearn.feature_extraction.text import TfidfVectorizer


#Unigram features
vectorizer_unigram=TfidfVectorizer()
vectorizer_unigram.fit(X_train)
tfidf_words=vectorizer_unigram.get_feature_names()
X_tfidf=vectorizer_unigram.transform(X)
X_train_tfidf=vectorizer_unigram.transform(X_train)
X_test_tfidf=vectorizer_unigram.transform(X_test)
X_CV_tfidf=vectorizer_unigram.transform(preprocessed_test)
print(X_train_tfidf.shape)

#Bigram features
vectorizer_bigram=TfidfVectorizer(ngram_range=(1,2), min_df=1)
vectorizer_bigram.fit(X_train)
tfidf_bigram_words=vectorizer_bigram.get_feature_names()
X_tfidf_bigram=vectorizer_bigram.transform(X)
X_train_tfidf_bigram=vectorizer_bigram.transform(X_train)
X_test_tfidf_bigram=vectorizer_bigram.transform(X_test)
X_CV_tfidf_bigram=vectorizer_bigram.transform(preprocessed_test)
print(X_train_tfidf_bigram.shape)

#Trigram features
vectorizer_trigram=TfidfVectorizer(ngram_range=(1,3), min_df=2)
vectorizer_trigram.fit(X_train)
X_train_tfidf_trigram=vectorizer_trigram.transform(X_train)
X_test_tfidf_trigram=vectorizer_trigram.transform(X_test)
X_CV_tfidf_trigram=vectorizer_trigram.transform(preprocessed_test)
print(X_train_tfidf_trigram.shape)

#Tetragram features
vectorizer_tetragram=TfidfVectorizer(ngram_range=(1,4), min_df=2)
vectorizer_tetragram.fit(X_train)
X_train_tfidf_tetragram=vectorizer_tetragram.transform(X_train)
X_test_tfidf_tetragram=vectorizer_tetragram.transform(X_test)
X_CV_tfidf_tetragram=vectorizer_tetragram.transform(preprocessed_test)
print(X_train_tfidf_tetragram.shape)

#Pentagram features
#Tetragram features
vectorizer_pentagram=TfidfVectorizer(ngram_range=(1,5), min_df=2)
vectorizer_pentagram.fit(X_train)
X_train_tfidf_pentagram=vectorizer_pentagram.transform(X_train)
X_test_tfidf_pentagram=vectorizer_pentagram.transform(X_test)
X_CV_tfidf_pentagram=vectorizer_pentagram.transform(preprocessed_test)
print(X_train_tfidf_pentagram.shape)

(21414, 26903)
(21414, 132046)
(21414, 28869)
(21414, 33386)
(21414, 36873)


In [231]:
ss_tr=[]
ss_data=[]
for i in X:
    s=sid.polarity_scores(i)
    ss_data.append(list(s.values()))
for i in X_train:
    s=sid.polarity_scores(i)
    ss_tr.append(list(s.values()))
ss_test=[]
for i in X_test:
    s=sid.polarity_scores(i)
    ss_test.append(list(s.values()))
ss_cv=[]
for i in test_tweet:
    s=sid.polarity_scores(i)
    ss_cv.append(list(s.values()))

In [232]:
from scipy.sparse import hstack
#X_test_grade
import scipy
data=hstack((X_tfidf,X_tfidf_bigram,ss_data)).tocsr()
X_tr=hstack((X_train_tfidf,X_train_tfidf_bigram,ss_tr)).tocsr()
X_cv=hstack((X_CV_tfidf,X_CV_tfidf_bigram,ss_cv)).tocsr()
X_tst=hstack((X_test_tfidf,X_test_tfidf_bigram,ss_test)).tocsr()
print(data.shape,X_tr.shape,X_tst.shape,X_cv.shape)

(31962, 158953) (21414, 158953) (10548, 158953) (17197, 158953)


**Using LinearSVC Algorithm**

In [228]:
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score
C=[0.001,0.01,0.1,0.5,1,10,100,1000]
for i in C:
  model = LinearSVC(penalty='l2', C=i, dual=False, random_state=0, max_iter=1000)
  model.fit(X_tr,Y_train)
  y_pred_tr = model.predict(X_tr)
  y_pred_te = model.predict(X_tst)
  print("C: {}".format(i))
  print('Train Accuracy:', accuracy_score(y_pred_tr, Y_train))
  print("Train F1 Score: ", f1_score(y_pred_tr, Y_train))
  print('Test Accuracy:', accuracy_score(y_pred_te, Y_test))
  print("Test F1 Score: ", f1_score(y_pred_te, Y_test))

C: 0.001
Train Accuracy: 0.929858970766788
Train F1 Score:  0.0
Test Accuracy: 0.9298445202882063
Test F1 Score:  0.0
C: 0.01
Train Accuracy: 0.9348090034556832
Train F1 Score:  0.13613861386138612
Test Accuracy: 0.9347743648084945
Test F1 Score:  0.13131313131313133
C: 0.1
Train Accuracy: 0.9679181843653685
Train F1 Score:  0.7057815845824411
Test Accuracy: 0.9584755403868032
Test F1 Score:  0.5921787709497207
C: 0.5
Train Accuracy: 0.9992528252545064
Train F1 Score:  0.9946595460614153
Test Accuracy: 0.9663443306788017
Test F1 Score:  0.7082990961380444
C: 1
Train Accuracy: 0.9997198094704399
Train F1 Score:  0.9980013324450367
Test Accuracy: 0.9661547212741752
Test F1 Score:  0.7146282973621103
C: 10
Train Accuracy: 0.9998599047352199
Train F1 Score:  0.9990003332222593
Test Accuracy: 0.9642586272279106
Test F1 Score:  0.7093292212798766
C: 100
Train Accuracy: 0.9998599047352199
Train F1 Score:  0.9990003332222593
Test Accuracy: 0.9638794084186576
Test F1 Score:  0.7089381207028265


In [233]:
model = LinearSVC(penalty='l2', C=10, dual=False, random_state=0, max_iter=1000)
model.fit(data,Y)
y_pred_tr = model.predict(X_tr)
y_pred_te = model.predict(X_tst)
print("C: {}".format(i))
print('Train Accuracy:', accuracy_score(y_pred_tr, Y_train))
print("Train F1 Score: ", f1_score(y_pred_tr, Y_train))
print('Test Accuracy:', accuracy_score(y_pred_te, Y_test))
print("Test F1 Score: ", f1_score(y_pred_te, Y_test))

C: song "so glad" free download! #shoegaze #newmusic #newsong
Train Accuracy: 0.9997198094704399
Train F1 Score:  0.9979986657771848
Test Accuracy: 0.9997155858930603
Test F1 Score:  0.997968855788761


In [234]:
y_pred_cv=model.predict(X_cv)
y_pred_cv=pd.DataFrame(y_pred_cv)
y_pred_cv.to_csv('/content/drive/My Drive/Twitter Sentiment Analysis/test_predictions_linearSVC.csv',index=True)

**Using Decision Trees Classifier**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
depth=[1,5,10,50,100, 500, 1000]
samples_split=[5, 10, 20, 45, 75, 100, 135, 270, 500]
train_f1_score=[]
test_f1_score=[]
for i in depth:
    for j in samples_split:
        clf=DecisionTreeClassifier(max_depth=i,min_samples_split=j,class_weight='balanced',random_state=0)
        clf.fit(X_tr,Y_train)
        y_pred_tr=clf.predict(X_tr)
        y_pred_te=clf.predict(X_tst)
        train_f1=f1_score(Y_train,y_pred_tr)
        test_f1=f1_score(Y_test,y_pred_te)
        print("Depth: {} Samples_Split: {} Train F1 Score: {} Test F1 Score: {}".format(i,j,train_f1,test_f1))
        train_f1_score.append(train_f1)
        test_f1_score.append(test_f1)
    print('\n')

In [ ]:
from sklearn.tree import DecisionTreeClassifier
depth=[1,5,10,50,100, 500, 1000]
samples_split=[5, 10, 20, 45, 75, 100, 135, 270, 500]
train_f1_score=[]
test_f1_score=[]
for i in depth:
    for j in samples_split:
        clf=DecisionTreeClassifier(max_depth=i,min_samples_split=j,class_weight='balanced',random_state=0)
        clf.fit(X_tr,Y_train)
        y_pred_tr=clf.predict(X_tr)
        y_pred_te=clf.predict(X_tst)
        train_f1=f1_score(Y_train,y_pred_tr)
        test_f1=f1_score(Y_test,y_pred_te)
        print("Depth: {} Samples_Split: {} Train F1 Score: {} Test F1 Score: {}".format(i,j,train_f1,test_f1))
        train_f1_score.append(train_f1)
        test_f1_score.append(test_f1)
    print('\n')

In [ ]:
clf=DecisionTreeClassifier(max_depth=1000,min_samples_split=500,class_weight='balanced',random_state=0)
clf.fit(X_tr,Y_train)
y_pred_tr=clf.predict(X_tr)
y_pred_te=clf.predict(X_tst)
y_pred_cv=clf.predict(X_cv)
train_score=f1_score(Y_train,y_pred_tr)
test_score=f1_score(Y_test,y_pred_te)
#cv_score=f1_score(Y_test,y_pred_cv)
print(train_score,test_score)

**Part 3: Using BOW features**

In [222]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer() #in scikit-learn
count_vect.fit(X_train)

X_train_bow = count_vect.transform(X_train)
X_test_bow = count_vect.transform(X_test)
X_cv_bow = count_vect.transform(preprocessed_test)

In [223]:
X_tr=hstack((X_train_bow,ss_tr)).tocsr()
X_cv=hstack((X_cv_bow,ss_cv)).tocsr()
X_tst=hstack((X_test_bow,ss_test)).tocsr()
print(X_tr.shape,X_tst.shape,X_cv.shape)

(21414, 26942) (10548, 26942) (17197, 26942)


In [224]:
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score
C=[0.001,0.01,0.1,0.5,1,10,100,1000]
for i in C:
  model = LinearSVC(penalty='l2', C=i, dual=False, random_state=0, max_iter=1000)
  model.fit(X_train_bow,Y_train)
  y_pred_tr = model.predict(X_train_bow)
  y_pred_te = model.predict(X_test_bow)
  print("C: {}".format(i))
  print('Train Accuracy:', accuracy_score(y_pred_tr, Y_train))
  print("Train F1 Score: ", f1_score(y_pred_tr, Y_train))
  print('Test Accuracy:', accuracy_score(y_pred_te, Y_test))
  print("Test F1 Score: ", f1_score(y_pred_te, Y_test))

C: 0.001
Train Accuracy: 0.9300457644531614
Train F1 Score:  0.007947019867549669
Test Accuracy: 0.9298445202882063
Test F1 Score:  0.0
C: 0.01
Train Accuracy: 0.9516671336508826
Train F1 Score:  0.4868616757560733
Test Accuracy: 0.94871065604854
Test F1 Score:  0.44169246646026833
C: 0.1
Train Accuracy: 0.9860371719435883
Train F1 Score:  0.8899521531100479
Test Accuracy: 0.9635949943117179
Test F1 Score:  0.6740237691001698
C: 0.5
Train Accuracy: 0.9977117773419258
Train F1 Score:  0.9834515366430261
Test Accuracy: 0.9654910883579826
Test F1 Score:  0.7187017001545596
C: 1
Train Accuracy: 0.9987858410385729
Train F1 Score:  0.9912810194500337
Test Accuracy: 0.962836556693212
Test F1 Score:  0.7048192771084338
C: 10
Train Accuracy: 0.9994396189408798
Train F1 Score:  0.9959946595460614
Test Accuracy: 0.9534508911642018
Test F1 Score:  0.6587908269631689
C: 100
Train Accuracy: 0.9995797142056598
Train F1 Score:  0.996996996996997
Test Accuracy: 0.949089874857793
Test F1 Score:  0.63691

**Part 3: Using Tfidf Weighted Word2Vec features**

In [194]:
import pickle
with open('/content/drive/My Drive/glove_vectors', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())

In [195]:
dictionary = dict(zip(vectorizer_unigram.get_feature_names(), list(vectorizer_unigram.idf_)))

In [196]:
from tqdm import tqdm
tfidf_w2v_vectors_tr = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_train): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    tf_idf_weight =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] # getting the vector for each word
            # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_vectors_tr.append(vector)

100%|██████████| 21414/21414 [00:33<00:00, 634.78it/s]


In [197]:
tfidf_w2v_vectors_te = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_test): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    tf_idf_weight =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] # getting the vector for each word
            # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_vectors_te.append(vector)

100%|██████████| 10548/10548 [00:17<00:00, 610.55it/s]


In [198]:
from tqdm import tqdm

tfidf_w2v_vectors_cv = []; # the avg-w2v for each sentence/review is stored in this list
    
for sentence in tqdm(preprocessed_test): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    tf_idf_weight =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] # getting the vector for each word
            # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_vectors_cv.append(vector)

100%|██████████| 17197/17197 [00:28<00:00, 612.95it/s]


In [199]:
tfidf_w2v_vectors_tr=np.array(tfidf_w2v_vectors_tr)
tfidf_w2v_vectors_te=np.array(tfidf_w2v_vectors_te)
tfidf_w2v_vectors_cv=np.array(tfidf_w2v_vectors_cv)
print(tfidf_w2v_vectors_tr.shape,tfidf_w2v_vectors_te.shape)

(21414, 300) (10548, 300)


In [154]:
tfidf_w2v_vectors_tr.shape,len(ss_tr[0])

((21414, 300), 4)

In [201]:
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score
C=[0.001,0.01,0.1,0.5,1,10,100,1000]
for i in C:
  model = LinearSVC(penalty='l2', C=i, dual=False, random_state=0, max_iter=1000)
  model.fit(tfidf_w2v_vectors_tr,Y_train)
  y_pred_tr = model.predict(tfidf_w2v_vectors_tr)
  y_pred_te = model.predict(tfidf_w2v_vectors_te)
  print("C: {}".format(i))
  print('Train Accuracy:', accuracy_score(y_pred_tr, Y_train))
  print("Train F1 Score: ", f1_score(y_pred_tr, Y_train))
  print('Test Accuracy:', accuracy_score(y_pred_te, Y_test))
  print("Test F1 Score: ", f1_score(y_pred_te, Y_test))

C: 0.001
Train Accuracy: 0.9374241150649109
Train F1 Score:  0.24379232505643342
Test Accuracy: 0.9373340917709518
Test F1 Score:  0.2547914317925592
C: 0.01
Train Accuracy: 0.9425609414401793
Train F1 Score:  0.39048562933597625
Test Accuracy: 0.9416003033750474
Test F1 Score:  0.39130434782608686
C: 0.1
Train Accuracy: 0.9448958625198468
Train F1 Score:  0.44652908067542213
Test Accuracy: 0.9429275692074327
Test F1 Score:  0.4331450094161958
C: 0.5
Train Accuracy: 0.9453628467357803
Train F1 Score:  0.4563197026022304
Test Accuracy: 0.943022373909746
Test F1 Score:  0.43779232927970063
C: 1
Train Accuracy: 0.9454095451573736
Train F1 Score:  0.45653184565318455
Test Accuracy: 0.9428327645051194
Test F1 Score:  0.4369747899159664
C: 10
Train Accuracy: 0.9453628467357803
Train F1 Score:  0.4568245125348189
Test Accuracy: 0.9427379598028062
Test F1 Score:  0.43656716417910446
C: 100
Train Accuracy: 0.9453628467357803
Train F1 Score:  0.4568245125348189
Test Accuracy: 0.9427379598028062
